In [ ]:
import google.generativeai as genai
import PIL.Image
from pathlib import Path
import json
from google.colab import drive, userdata

In [ ]:
#connection to my google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# API key to google ai studio
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
print("API Key successfully set")

API Key successfully set


In [ ]:
#Defining paths
JSON_FILE_PATH = Path("/content/drive/MyDrive/ProjectSemester1/words_to_presentationsNew/presentations_words.json")
SLIDES_DIRECTORY = Path("/content/drive/MyDrive/ProjectSemester1/slides")

# Path to the output folder where I will save the summaries
OUTPUT_SUMMERIES_DIRECTORY = Path("/content/drive/MyDrive/ProjectSemester1/summaries")
#The model I will use
MODEL = genai.GenerativeModel('gemini-1.5-flash')

def load_presentations_words(json_file_path: Path):
  if not json_file_path.exists():
      print(f"Not found the file in the path {json_file_path}")
      exit()

  try:
      with json_file_path.open("r", encoding="utf-8") as f:
          return json.load(f)
      print(f"succeeded {json_file_path}")
  except json.JSONDecodeError:
      print(f"failed {json_file_path}")
      exit()


def summerize_presentation(output_summaries_directory: Path, json_file_path: Path, slides_directory: Path) -> None:
    # Make sure the output folder exists, if not create it
    output_summaries_directory.mkdir(parents=True, exist_ok=True)

    presentations_words = load_presentations_words(json_file_path)


    for i, slide_image_path in enumerate(slides_directory.iterdir()):
        slide_image_name = slide_image_path.name


        text_list_for_slide = presentations_words.get(slide_image_name, [])

        slide_summary = summerize_slide(slide_image_path, i, text_list_for_slide)

        summary_filename = f"summary_slide_{i}.txt"
        summary_file_path = output_summaries_directory / summary_filename
        with summary_file_path.open("w", encoding="utf-8") as f:
            f.write(slide_summary)
        print(f"The summary was successfully saved to: {summary_file_path}")

    print("The summary process has ended")


def summerize_slide(slide_image_path: Path, i: int, text_list_for_slide: list[str]) -> str:

    if not text_list_for_slide: # If the list is empty
        lecture_text_for_slide = "אין טקסט דיבור משויך לשקף זה. הסיכום יתבסס על התמונה בלבד."
    else:
        lecture_text_for_slide = ".".join(text_list_for_slide) #Join the texts into one string

    try:
        # Loading page image
        img = PIL.Image.open(slide_image_path)

        # Creating a form for the model
        prompt_parts = [
            img,
            f"""
            בהתבסס על התמונה של השקף שלפנייך ועל הטקסט שהוא כתובית של מה שנאמר בזמן הצגת העמוד בעת ההרצאה, בבקשה סכם בקצרה את הנקודות המרכזיות ורעיונות העיקריים של העמוד (שהוא חלק מההרצאה) כפי שהם משתקפים גם מהשקף וגם מהטקסט.
            חשוב הסיכום הוא לעמוד לא להרצאה כולה
            בנוסף חשוב שתחזיר רק סיכום כפלט, לא הקדמה או משהו אחר, רק את הסיכום!
            טקסט מההרצאה שנאמר בזמן הצגת העמוד הרלוונטי(עשוי להכיל רעשים ורעשי רקע):
            "{lecture_text_for_slide}"
            """,
        ]
        #The result of the summary generated by the model
        response = MODEL.generate_content(prompt_parts)
        slide_summary = response.text

        return slide_summary

    except Exception as e:
        print(f"Error processing slide {i}: {e}. Skipping this slide.")

summerize_presentation(OUTPUT_SUMMERIES_DIRECTORY, JSON_FILE_PATH, SLIDES_DIRECTORY)

תמונה נטענה בהצלחה מ: /content/drive/MyDrive/ProjectSemester1/slides/slide_1.png
The summary was successfully saved to: /content/drive/MyDrive/ProjectSemester1/summaries/summary_slide_0.txt
תמונה נטענה בהצלחה מ: /content/drive/MyDrive/ProjectSemester1/slides/slide_2.png
The summary was successfully saved to: /content/drive/MyDrive/ProjectSemester1/summaries/summary_slide_1.txt
תמונה נטענה בהצלחה מ: /content/drive/MyDrive/ProjectSemester1/slides/slide_3.png
The summary was successfully saved to: /content/drive/MyDrive/ProjectSemester1/summaries/summary_slide_2.txt
תמונה נטענה בהצלחה מ: /content/drive/MyDrive/ProjectSemester1/slides/slide_4.png
The summary was successfully saved to: /content/drive/MyDrive/ProjectSemester1/summaries/summary_slide_3.txt
תמונה נטענה בהצלחה מ: /content/drive/MyDrive/ProjectSemester1/slides/slide_5.png
The summary was successfully saved to: /content/drive/MyDrive/ProjectSemester1/summaries/summary_slide_4.txt
תמונה נטענה בהצלחה מ: /content/drive/MyDrive/Proje